In [40]:
import hashlib
import json
import time
import random
from typing import List, Dict, Optional

In [41]:
"""
Кошелек для хранения ключей и создания транзакций
"""
class Wallet:
    def __init__(self, name=""):
        self.name = name
        self.private_key = hashlib.sha256(str(time.time() + random.random()).encode()).hexdigest()
        self.public_key = hashlib.sha256(self.private_key.encode()).hexdigest()
    
    def create_transaction(self, to_address, amount):
        return Transaction(self.public_key, to_address, amount)

In [42]:
"""
Транзакция биткоина
"""
class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.timestamp = time.time()
        self.signature = self._generate_signature()
    
    def _generate_signature(self):
        data = f"{self.from_address}{self.to_address}{self.amount}{self.timestamp}"
        return hashlib.sha256(data.encode()).hexdigest()
    
    def to_dict(self):
        return {
            'from_address': self.from_address,
            'to_address': self.to_address,
            'amount': self.amount,
            'timestamp': self.timestamp,
            'signature': self.signature
        }

In [43]:
"""
Блок в блокчейне
"""
class Block:
    def __init__(self, transactions, previous_hash):
        self.timestamp = time.time()
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()
    
    def calculate_hash(self):
        block_data = {
            'timestamp': self.timestamp,
            'transactions': [tx.to_dict() for tx in self.transactions],
            'previous_hash': self.previous_hash,
            'nonce': self.nonce
        }
        return hashlib.sha256(json.dumps(block_data, sort_keys=True).encode()).hexdigest()
    
    def mine_block(self, difficulty):
        target = '0' * difficulty
        while self.hash[:difficulty] != target:
            self.nonce += 1
            self.hash = self.calculate_hash()
    
    def to_dict(self):
        return {
            'hash': self.hash,
            'previous_hash': self.previous_hash,
            'timestamp': self.timestamp,
            'nonce': self.nonce,
            'transactions': [tx.to_dict() for tx in self.transactions]
        }


In [44]:
"""
Блокчейн
"""
class Blockchain:
    def __init__(self):
        self.chain: List[Block] = [self.create_genesis_block()]
        self.difficulty = 2
        self.pending_transactions: List[Transaction] = []
        self.mining_reward = 100
        self.initial_balances = {}
    
    def create_genesis_block(self):
        genesis_transactions = []
        return Block(genesis_transactions, "0")
    
    def initialize_balances(self, wallets_with_balances):
        self.initial_balances = wallets_with_balances
    
    def get_latest_block(self):
        return self.chain[-1]
    
    def add_transaction(self, transaction):
        if transaction.from_address and self.get_balance(transaction.from_address) < transaction.amount:
            print(f"Ошибка: Недостаточно средств у отправителя. Баланс: {self.get_balance(transaction.from_address)}, требуется: {transaction.amount}")
            return False
        
        self.pending_transactions.append(transaction)
        print(f"Транзакция добавлена: {transaction.from_address}... -> {transaction.to_address}...: {transaction.amount}")
        return True
    
    def mine_pending_transactions(self, mining_reward_address):
        if not self.pending_transactions:
            print("Нет транзакций для майнинга")
            return
        
        block = Block(self.pending_transactions, self.get_latest_block().hash)
        block.mine_block(self.difficulty)
        
        self.chain.append(block)
        print(f"Блок успешно добыт! Хэш: {block.hash}...")
        
        reward_transaction = Transaction(None, mining_reward_address, self.mining_reward)
        self.pending_transactions = [reward_transaction]
        print(f"Награда за майнинг: {self.mining_reward} -> {mining_reward_address}...")

    def get_balance(self, address):
        balance = 0
        
        if address in self.initial_balances:
            balance += self.initial_balances[address]
        
        for block in self.chain:
            for transaction in block.transactions:
                if transaction.from_address == address:
                    balance -= transaction.amount
                if transaction.to_address == address:
                    balance += transaction.amount
        
        for transaction in self.pending_transactions:
            if transaction.from_address == address:
                balance -= transaction.amount
            if transaction.to_address == address:
                balance += transaction.amount
        
        return balance
    
    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i-1]
            
            if current_block.hash != current_block.calculate_hash():
                return False
            
            if current_block.previous_hash != previous_block.hash:
                return False
        
        return True
    
    def print_chain(self):
        for i, block in enumerate(self.chain):
            print(f"Блок {i}:")
            print(f"  Хэш: {block.hash}...")
            print(f"  Предыдущий хэш: {block.previous_hash}...")
            print(f"  Nonce: {block.nonce}")
            print(f"  Количество транзакций: {len(block.transactions)}")
            if block.transactions:
                print("  Транзакции:")
                for tx in block.transactions:
                    from_addr = tx.from_address + "..." if tx.from_address else "Система"
                    to_addr = tx.to_address + "..." if tx.to_address else "Неизвестно"
                    print(f"    {from_addr} -> {to_addr}: {tx.amount}")
            print("-" * 50)

In [45]:
print("=" * 60)
print("ДЕМОНСТРАЦИЯ РАБОТЫ БЛОКЧЕЙНА")
print("=" * 60)

bitcoin = Blockchain()

anya = Wallet("Аня")
igor = Wallet("Игорь")
olga = Wallet("Ольга")
miner = Wallet("Майнер")

print("\nСозданы кошельки:")
print(f"  Аня: {anya.public_key}...")
print(f"  Игорь: {igor.public_key}...")
print(f"  Ольга: {olga.public_key}...")
print(f"  Майнер: {miner.public_key}...")

initial_balances = {
    anya.public_key: 500,
    igor.public_key: 300,
    olga.public_key: 200
}
bitcoin.initialize_balances(initial_balances)

print("\nНачальные балансы:")
print(f"  Аня: {bitcoin.get_balance(anya.public_key)}")
print(f"  Игорь: {bitcoin.get_balance(igor.public_key)}")
print(f"  Ольга: {bitcoin.get_balance(olga.public_key)}")
print(f"  Майнер: {bitcoin.get_balance(miner.public_key)}")

print("\n1. Создаем транзакцию")
tx1 = anya.create_transaction(igor.public_key, 50)
bitcoin.add_transaction(tx1)

tx2 = igor.create_transaction(olga.public_key, 30)
bitcoin.add_transaction(tx2)

tx3 = olga.create_transaction(anya.public_key, 20)
bitcoin.add_transaction(tx3)

print("\n2. Майним первый блок...")
bitcoin.mine_pending_transactions(miner.public_key)

print("\n3. Создаем транзакцию")
tx4 = anya.create_transaction(olga.public_key, 40)
bitcoin.add_transaction(tx4)

tx5 = igor.create_transaction(anya.public_key, 25)
bitcoin.add_transaction(tx5)

print("\n4. Майним второй блок")
bitcoin.mine_pending_transactions(miner.public_key)

print("\n5. Транзакция без достаточного баланса")
tx6 = olga.create_transaction(anya.public_key, 500)
bitcoin.add_transaction(tx6)

print("\n" + "=" * 60)
print("ИНФОРМАЦИЯ О БЛОКЧЕЙНЕ:")
print("=" * 60)
bitcoin.print_chain()

print("\n" + "=" * 60)
print("ФИНАЛЬНЫЕ БАЛАНСЫ:")
print("=" * 60)
print(f"Аня: {bitcoin.get_balance(anya.public_key)}")
print(f"Игорь: {bitcoin.get_balance(igor.public_key)}")
print(f"Ольга: {bitcoin.get_balance(olga.public_key)}")
print(f"Майнер: {bitcoin.get_balance(miner.public_key)}")

print(f"\nБлокчейн валиден: {bitcoin.is_chain_valid()}")

ДЕМОНСТРАЦИЯ РАБОТЫ БЛОКЧЕЙНА

Созданы кошельки:
  Аня: 14c26cfd5a5c5ea827515f070c1c19691b01455c454e7ffcd34f89869b7ecc76...
  Игорь: 0f52297f99a127fe6be54f09b75cb8996e87530fb60f88d7d77a4992ab2ed612...
  Ольга: a65902e57bac8a983f28e88b1d461e6e2b7b450102ba02b24d12f1a4a005e956...
  Майнер: 5d2354e218887397b443118a731f59e276af1dcad9eef0c1749734e67491bf5a...

Начальные балансы:
  Аня: 500
  Игорь: 300
  Ольга: 200
  Майнер: 0

1. Создаем транзакцию
Транзакция добавлена: 14c26cfd5a5c5ea827515f070c1c19691b01455c454e7ffcd34f89869b7ecc76... -> 0f52297f99a127fe6be54f09b75cb8996e87530fb60f88d7d77a4992ab2ed612...: 50
Транзакция добавлена: 0f52297f99a127fe6be54f09b75cb8996e87530fb60f88d7d77a4992ab2ed612... -> a65902e57bac8a983f28e88b1d461e6e2b7b450102ba02b24d12f1a4a005e956...: 30
Транзакция добавлена: a65902e57bac8a983f28e88b1d461e6e2b7b450102ba02b24d12f1a4a005e956... -> 14c26cfd5a5c5ea827515f070c1c19691b01455c454e7ffcd34f89869b7ecc76...: 20

2. Майним первый блок...
Блок успешно добыт! Хэш: 00b444